In [1]:
import sys
import os
import matplotlib
import PIL
import six
import numpy as np
import math
import time
import paddle
import paddle.fluid as fluid

matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
def plot(filename, gen_data):
    pad_dim = 1
    paded = pad_dim + img_dim
    gen_data = gen_data.reshape(gen_data.shape[0], img_dim, img_dim)
    n = int(math.ceil(math.sqrt(gen_data.shape[0])))
    gen_data = (np.pad(
        gen_data, [[0, n * n - gen_data.shape[0]], [pad_dim, 0], [pad_dim, 0]],
        'constant').reshape((n, n, paded, paded)).transpose((0, 2, 1, 3))
                .reshape((n * paded, n * paded)))
    fig = plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.imsave(filename, gen_data, cmap='Greys_r', vmin=-1, vmax=1)
    return fig

gf_dim = 64 # the number of basic channels of the generator's feature map. The number of all the channels of feature maps in the generator is a multiple of the number of basic channels
df_dim = 64 # the number of basic channels of the discriminator's feature map. The number of all the channels of feature maps in the discriminator is a multiple of the number of basic channels
gfc_dim = 1024 * 2 # the dimension of full connection layer of generator
dfc_dim = 1024 # the dimension of full connection layer of discriminator
img_dim = 28  # size of the input picture

NOISE_SIZE = 100  # dimension of input noise
LEARNING_RATE = 2e-4 # learning rate of training

epoch = 20         # epoch number of training
output = "./output_dcgan"   # storage path of model and test results
use_cudnn = False  # use cuDNN or not
use_gpu=True       # use GPU or not

def bn(x, name=None, act='relu'):
    return fluid.layers.batch_norm(
        x,
        param_attr=name + '1',
        bias_attr=name + '2',
        moving_mean_name=name + '3',
        moving_variance_name=name + '4',
        name=name,
        act=act)

def conv(x, num_filters, name=None, act=None):
    return fluid.nets.simple_img_conv_pool(
        input=x,
        filter_size=5,
        num_filters=num_filters,
        pool_size=2,
        pool_stride=2,
        param_attr=name + 'w',
        bias_attr=name + 'b',
        use_cudnn=use_cudnn,
        act=act)

def fc(x, num_filters, name=None, act=None):
    return fluid.layers.fc(input=x,
                           size=num_filters,
                           act=act,
                           param_attr=name + 'w',
                           bias_attr=name + 'b')
def deconv(x,
           num_filters,
           name=None,
           filter_size=5,
           stride=2,
           dilation=1,
           padding=2,
           output_size=None,
           act=None):
    return fluid.layers.conv2d_transpose(
        input=x,
        param_attr=name + 'w',
        bias_attr=name + 'b',
        num_filters=num_filters,
        output_size=output_size,
        filter_size=filter_size,
        stride=stride,
        dilation=dilation,
        padding=padding,
        use_cudnn=use_cudnn,
        act=act)

In [3]:
def D(x, num_classes):
    x = fluid.layers.reshape(x=x, shape=[-1, 1, 28, 28])
    x = conv(x, df_dim, act='leaky_relu',name='conv1')
    x = bn(conv(x, df_dim * 2,name='conv2'), act='leaky_relu',name='bn1')
    x = bn(fc(x, dfc_dim,name='fc1'), act='leaky_relu',name='bn2')
    x = fc(x, num_classes, act='sigmoid',name='fc2')
    return x

In [4]:
def G(x):
    x = bn(fc(x, gfc_dim,name='fc3'),name='bn3')
    x = bn(fc(x, gf_dim * 2 * img_dim // 4 * img_dim // 4,name='fc4'),name='bn4')
    x = fluid.layers.reshape(x, [-1, gf_dim * 2, img_dim // 4, img_dim // 4])
    x = deconv(x, gf_dim * 2, act='relu', output_size=[14, 14],name='deconv1')
    x = deconv(x, num_filters=1, filter_size=5, padding=2, act='tanh', output_size=[28, 28],name='deconv2')
    x = fluid.layers.reshape(x, shape=[-1, 28 * 28])
    return x

In [5]:
def loss(x, label):
    return fluid.layers.mean(
        fluid.layers.sigmoid_cross_entropy_with_logits(x=x, label=label))

In [6]:
paddle.enable_static()

In [7]:
d_program = fluid.Program()
dg_program = fluid.Program()

# Define the program to distinguish the real picture
with fluid.program_guard(d_program):
    # size of the input picture is28*28=784
    img = fluid.data(name='img', shape=[None, 784], dtype='float32')
    # label shape=1
    label = fluid.data(name='label', shape=[None, 1], dtype='float32')
    d_logit = D(img, 1)
    d_loss = loss(d_logit, label)

# Define the program to distinguish the generated pictures
with fluid.program_guard(dg_program):
    noise = fluid.data(
        name='noise', shape=[None, NOISE_SIZE], dtype='float32')
    # Noise data as input to generate image
    g_img = G(x=noise)

    g_program = dg_program.clone()
    g_program_test = dg_program.clone(for_test=True)

    # Judge the probability that the generated image is a real sample
    dg_logit = D(g_img, 1)

    # Calculate the loss of the generated image as the real sample
    noise_shape = fluid.layers.shape(noise)
    dg_loss = loss(
        dg_logit,
        fluid.layers.fill_constant(
            dtype='float32', shape=[noise_shape[0], 1], value=1.0))

In [8]:
opt = fluid.optimizer.Adam(learning_rate=LEARNING_RATE)
opt.minimize(loss=d_loss)
parameters = [p.name for p in g_program.global_block().all_parameters()]
opt.minimize(loss=dg_loss, parameter_list=parameters)

([{Beta1PowOut=['bn31_beta1_pow_acc_0'], Beta2PowOut=['bn31_beta2_pow_acc_0'], Moment1Out=['bn31_moment1_0'], Moment2Out=['bn31_moment2_0'], ParamOut=['bn31']} = adam(inputs={Beta1Pow=['bn31_beta1_pow_acc_0'], Beta1Tensor=[], Beta2Pow=['bn31_beta2_pow_acc_0'], Beta2Tensor=[], Grad=['bn31@GRAD'], LearningRate=['learning_rate_1'], MasterParam=[], Moment1=['bn31_moment1_0'], Moment2=['bn31_moment2_0'], Param=['bn31']}, beta1 = 0.8999999761581421, beta2 = 0.9990000128746033, epsilon = 9.99999993922529e-09, lazy_mode = False, min_row_size_to_use_multithread = 1000, multi_precision = False, op_device = , op_namescope = /optimizer_12/, op_role = 2, op_role_var = ['bn31', 'bn31@GRAD']),
  {Beta1PowOut=['bn32_beta1_pow_acc_0'], Beta2PowOut=['bn32_beta2_pow_acc_0'], Moment1Out=['bn32_moment1_0'], Moment2Out=['bn32_moment2_0'], ParamOut=['bn32']} = adam(inputs={Beta1Pow=['bn32_beta1_pow_acc_0'], Beta1Tensor=[], Beta2Pow=['bn32_beta2_pow_acc_0'], Beta2Tensor=[], Grad=['bn32@GRAD'], LearningRate=['

In [9]:
batch_size = 128   # Minibatch size

train_reader = fluid.io.batch(
    fluid.io.shuffle(
        paddle.dataset.mnist.train(), buf_size=60000),
    batch_size=batch_size)

/home/khoa/miniconda3/envs/bd_lira/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
API "paddle.dataset.mnist.train" is deprecated since 2.0.0, and will be removed in future versions. Please use "paddle.vision.datasets.MNIST" instead.
reason: Please use new dataset API which supports paddle.io.DataLoader 
  """


In [10]:
print(f'Using {"GPU" if use_gpu else "CPU"}')
if use_gpu:
    exe = fluid.Executor(fluid.CUDAPlace(0))
else:
    exe = fluid.Executor(fluid.CPUPlace())

exe.run(fluid.default_startup_program())

Using GPU


W0816 11:32:27.491343 3416898 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 11.2, Runtime API Version: 10.2
W0816 11:32:27.491688 3416898 dynamic_loader.cc:267] The third-party dynamic library (libcudnn.so) that Paddle depends on is not configured correctly. (error code is libcudnn.so: cannot open shared object file: No such file or directory)
  Suggestions:
  1. Check if the third-party dynamic library (e.g. CUDA, CUDNN) is installed correctly and its version is matched with paddlepaddle you installed.
  2. Configure third-party dynamic library environment variables as follows:
  - Linux: set LD_LIBRARY_PATH by `export LD_LIBRARY_PATH=...`
  - Windows: set PATH by `set PATH=XXX;


RuntimeError: (PreconditionNotMet) Cannot load cudnn shared library. Cannot invoke method cudnnGetVersion.
  [Hint: cudnn_dso_handle should not be null.] (at /paddle/paddle/fluid/platform/dynload/cudnn.cc:59)


In [12]:
t_time = 0
losses = [[], []]

# The number of iterations of the discriminator
NUM_TRAIN_TIMES_OF_DG = 2  

# Noise data of final generated image
const_n = np.random.uniform(
    low=-1.0, high=1.0,
    size=[batch_size, NOISE_SIZE]).astype('float32')

for pass_id in range(epoch):
    for batch_id, data in enumerate(train_reader()):
        if len(data) != batch_size:
            continue

        # Generating noise data during training
        noise_data = np.random.uniform(
            low=-1.0, high=1.0,
            size=[batch_size, NOISE_SIZE]).astype('float32')

        # Real image
        real_image = np.array(list(map(lambda x: x[0], data))).reshape(
            -1, 784).astype('float32')
        # Real label
        real_labels = np.ones(
            shape=[real_image.shape[0], 1], dtype='float32')
        # Fake label
        fake_labels = np.zeros(
            shape=[real_image.shape[0], 1], dtype='float32')
        total_label = np.concatenate([real_labels, fake_labels])
        s_time = time.time()

        # Fake image
        generated_image = exe.run(g_program,
                                  feed={'noise': noise_data},
                                  fetch_list=[g_img])[0]

        total_images = np.concatenate([real_image, generated_image])

        # D loss of judging fake pictures as fake
        d_loss_1 = exe.run(d_program,
                           feed={
                               'img': generated_image,
                               'label': fake_labels,
                           },
                           fetch_list=[d_loss])[0][0]

        # D loss of judging true pictures as true
        d_loss_2 = exe.run(d_program,
                           feed={
                               'img': real_image,
                               'label': real_labels,
                           },
                           fetch_list=[d_loss])[0][0]

        d_loss_n = d_loss_1 + d_loss_2
        losses[0].append(d_loss_n)

        # Training generator
        for _ in six.moves.xrange(NUM_TRAIN_TIMES_OF_DG):
            noise_data = np.random.uniform(
                low=-1.0, high=1.0,
                size=[batch_size, NOISE_SIZE]).astype('float32')
            dg_loss_n = exe.run(dg_program,
                                 feed={'noise': noise_data},
                                 fetch_list=[dg_loss])[0][0]
            losses[1].append(dg_loss_n)
        t_time += (time.time() - s_time)
        if batch_id % 10 == 0 :
            if not os.path.exists(output):
                os.makedirs(output)
            # Results of each round
            generated_images = exe.run(g_program_test,
                                       feed={'noise': const_n},
                                       fetch_list=[g_img])[0]
            # Connect real pictures to generated pictures
            total_images = np.concatenate([real_image, generated_images])
            fig = plot(total_images)
            msg = "Epoch ID={0} Batch ID={1} D-Loss={2} DG-Loss={3}\n ".format(
                pass_id, batch_id,
                d_loss_n, dg_loss_n)
            print(msg)
            plt.title(msg)
            plt.savefig(
                '{}/{:04d}_{:04d}.png'.format(output, pass_id,
                                              batch_id),
                bbox_inches='tight')
            plt.close(fig)

NameError: name 'g_program' is not defined

In [ ]:
def display_image(epoch_no,batch_id):
  return PIL.Image.open('output_dcgan/{:04d}_{:04d}.png'.format(epoch_no,batch_id))

# Observe the generated images of the 10th epoch and 460 batches:
display_image(10,460)